# 🚀 Multi-Agent Systems & Workflow Patterns

**Welcome to the Kaggle 5-day Agents course!**

In the previous notebook, you built a **single agent** that could take action. Now, you'll learn how to scale up by building **agent teams**.

Just like a team of people, you can create specialized agents that collaborate to solve complex problems. This is called a **multi-agent system**, and it's one of the most powerful concepts in AI agent development.

In this notebook, you'll:

- ✅ Learn when to use multi-agent systems in [Agent Development Kit (ADK)](https://google.github.io/adk-docs/)
- ✅ Build your first system using an LLM as a "manager"
- ✅ Learn three core workflow patterns (Sequential, Parallel, and Loop) to coordinate your agent teams

**ℹ️ Note: No submission required!**

This notebook is for your hands-on practice and learning only. You **do not** need to submit it anywhere to complete the course.

## 📖 Get started with Kaggle Notebooks

If this is your first time using Kaggle Notebooks, welcome! You can learn more about using Kaggle Notebooks [in the documentation](https://www.kaggle.com/docs/notebooks).

Here's how to get started:

**1. Verify Your Account (Required)**

To use the Kaggle Notebooks in this course, you'll need to verify your account with a phone number.

You can do this in your [Kaggle settings](https://www.kaggle.com/settings).

**2. Make Your Own Copy**

To run any code in this notebook, you first need your own editable copy.

Click the `Copy and Edit` button in the top-right corner.

![Copy and Edit button](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_1.png)

This creates a private copy of the notebook just for you.

**3. Run Code Cells**

Once you have your copy, you can run code.

Click the ▶️ Run button next to any code cell to execute it.

![Run cell button](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_2.png)

Run the cells in order from top to bottom.

**4. If You Get Stuck**

To restart: Select `Factory reset` from the `Run` menu.

For help: Ask questions on the [Kaggle Discord](https://discord.com/invite/kaggle) server.

### Section 1

## ⚙️ Setup

### Install dependencies

The Kaggle Notebooks environment includes a pre-installed version of the [google-adk](https://google.github.io/adk-docs/) library for Python and its required dependencies, so you don't need to install additional packages in this notebook.

To install and use ADK in your own Python development environment outside of this course, you can do so by running:

```
pip install google-adk
```

In [1]:
pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 5.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich

### 1.1 Configure your Gemini API Key

This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/docs), which requires authentication.

**1. Get your API key**

If you don't have one already, create an [API key in Google AI Studio](https://aistudio.google.com/app/api-keys).

**2. Add the key to Kaggle Secrets**

Next, you will need to add your API key to your Kaggle Notebook as a Kaggle User Secret.

1. In the top menu bar of the notebook editor, select `Add-ons` then `Secrets`.
2. Create a new secret with the label `GOOGLE_API_KEY`.
3. Paste your API key into the "Value" field and click "Save".
4. Ensure that the checkbox next to `GOOGLE_API_KEY` is selected so that the secret is attached to the notebook.

**3. Authenticate in the notebook**

Run the cell below to complete authentication.

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

✅ Gemini API key setup complete.


### 1.2 Import ADK components

Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [3]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


---
### Section 2

## 🤔 Why Multi-Agent Systems? + Your First Multi-Agent

**The Problem: The "Do-It-All" Agent**

Single agents can do a lot. But what happens when the task gets complex? A single "monolithic" agent that tries to do research, writing, editing, and fact-checking all at once becomes a problem. Its instruction prompt gets long and confusing. It's hard to debug (which part failed?), difficult to maintain, and often produces unreliable results.

**The Solution: A Team of Specialists**

Instead of one "do-it-all" agent, we can build a **multi-agent system**. This is a team of simple, specialized agents that collaborate, just like a real-world team. Each agent has one clear job (e.g., one agent *only* does research, another *only* writes). This makes them easier to build, easier to test, and much more powerful and reliable when working together.

To learn more, check out the documentation related to [LLM agents in ADK](https://google.github.io/adk-docs/agents/llm-agents/).

**Architecture: Single Agent vs Multi-Agent Team**

<!--
```mermaid
graph TD
    subgraph Single["❌ Monolithic Agent"]
        A["One Agent Does Everything"]
    end

    subgraph Multi["✅ Multi-Agent Team"]
        B["Root Coordinator"] -- > C["Research Specialist"]
        B -- > E["Summary Specialist"]

        C -- >|findings| F["Shared State"]
        E -- >|summary| F
    end

    style A fill:#ffcccc
    style B fill:#ccffcc
    style F fill:#ffffcc
```
-->

<img width="800" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/multi-agent-team.png" alt="Multi-agent Team" />

### 2.1 Example: Research & Summarization System

Let's build a system with two specialized agents:

1. **Research Agent** - Searches for information using Google Search
2. **Summarizer Agent** - Creates concise summaries from research findings

In [4]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings", # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

✅ research_agent created.


In [5]:
# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model="gemini-2.5-flash-lite",
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


Refer to the ADK documentation for more information on [guiding agents with clear and specific instructions](https://google.github.io/adk-docs/agents/llm-agents/).

Then we bring the agents together under a root agent, or coordinator:

In [6]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model="gemini-2.5-flash-lite",
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[
        AgentTool(research_agent),
        AgentTool(summarizer_agent)
    ],
)

print("✅ root_agent created.")

✅ root_agent created.


Here we're using `AgentTool` to wrap the sub-agents to make them callable tools for the root agent. We'll explore `AgentTool` in-detail on Day 2.

Let's run the agent and ask it about a topic:

In [7]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("What are the latest advancements in security operations and how the paradigm shift is happening from Security Analyst to AI security architect or AI security engineer or AI threat modeller?")


 ### Created new session: debug_session_id

User > What are the latest advancements in security operations and how the paradigm shift is happening from Security Analyst to AI security architect or AI security engineer or AI threat modeller?


ResearchCoordinator > The latest advancements in security operations are characterized by the increasing integration of Artificial Intelligence (AI). This includes AI-powered surveillance with predictive capabilities, the emergence of autonomous \"Agentic SOCs\" that automate threat detection and response, and the adoption of flexible hybrid security architectures. Security systems are also increasingly being utilized as sources of business intelligence.

This evolving landscape is driving a paradigm shift from traditional Security Analyst roles to specialized AI security positions. Key new roles include:

*   **AI Security Architect:** Responsible for establishing governance structures, policies, and risk management for AI deployments, ensuring compliance with regulations and ethical standards, and guiding the secure design of AI platforms.
*   **AI Security Engineer:** Focuses on the practical implementation and testing of security solutions within AI environments. This involves safe



---

### Section 3
## 🚥 Sequential Workflows - The Assembly Line

**The Problem: Unpredictable Order**

The previous multi-agent system worked, but it relied on a **detailed instruction prompt** to force the LLM to run steps in order. This can be unreliable. A complex LLM might decide to skip a step, run them in the wrong order, or get "stuck," making the process unpredictable.

**The Solution: A Fixed Pipeline**

When you need tasks to happen in a **guaranteed, specific order**, you can use a `SequentialAgent`. This agent acts like an assembly line, running each sub-agent in the exact order you list them. The output of one agent automatically becomes the input for the next, creating a predictable and reliable workflow.

**Use Sequential when:** Order matters, you need a linear pipeline, or each step builds on the previous one.

To learn more, check out the documentation related to [sequential agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/).

**Architecture: Blog Post Creation Pipeline**

<!--
```mermaid
graph LR
    A["User Input: Blog about AI"] -- > B["Outline Agent"]
    B -- >|blog_outline| C["Writer Agent"]
    C -- >|blog_draft| D["Editor Agent"]
    D -- >|final_blog| E["Output"]

    style B fill:#ffcccc
    style C fill:#ccffcc
    style D fill:#ccccff
```
-->

<img width="1000" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/sequential-agent.png" alt="Sequential Agent" />

### 3.1 Example: Blog Post Creation with Sequential Agents

Let's build a system with three specialized agents:

1. **Outline Agent** - Creates a blog outline for a given topic
2. **Writer Agent** - Writes a blog post
3. **Editor Agent** - Edits a blog post draft for clarity and structure

In [8]:
# Outline Agent: Creates the initial blog post outline.
outline_agent = Agent(
    name="OutlineAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline", # The result of this agent will be stored in the session state with this key.
)

print("✅ outline_agent created.")

✅ outline_agent created.


In [9]:
# Writer Agent: Writes the full blog post based on the outline from the previous agent.
writer_agent = Agent(
    name="WriterAgent",
    model="gemini-2.5-flash-lite",
    # The `{blog_outline}` placeholder automatically injects the state value from the previous agent's output.
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft", # The result of this agent will be stored with this key.
)

print("✅ writer_agent created.")

✅ writer_agent created.


In [10]:
# Editor Agent: Edits and polishes the draft from the writer agent.
editor_agent = Agent(
    name="EditorAgent",
    model="gemini-2.5-flash-lite",
    # This agent receives the `{blog_draft}` from the writer agent's output.
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog", # This is the final output of the entire pipeline.
)

print("✅ editor_agent created.")

✅ editor_agent created.


Then we bring the agents together under a sequential agent, which runs the agents in the order that they are listed:

In [11]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


Let's run the agent and give it a topic to write a blog post about:

In [ ]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a blog post about the recent evolution of SOC and being a SOC engineer how to achieve a high income steam in 2026?")



User > Write a blog post about the recent evolution of SOC and being a SOC engineer how to achieve a high income steam in 2026?
OutlineAgent > ## OutlineAgent's Blog Post Outline:

**Headline:** **Level Up Your SOC Skills: The High-Income Blueprint for Cybersecurity Engineers in 2026**

**Introduction Hook:** The cybersecurity landscape is a battlefield, and the Security Operations Center (SOC) is on the front lines. But the tools and tactics of yesterday are no longer enough. As threats become more sophisticated, so too must the professionals defending our digital fortresses. If you're a SOC engineer looking to not just survive, but thrive and command a high income in 2026, you need to understand the seismic shifts happening in SOC evolution and strategically position yourself for success.

**Main Section 1: The Evolving SOC: Beyond the Alert Triage**

*   **From Reactive to Proactive & Predictive:** Discuss the shift from simply responding to alerts to actively hunting for threats and predicting future attack vectors using threat intelligence and advanced analytics.
*   **The Rise of Automation and AI/ML:** Explore how AI and Machine Learning are transforming SOC efficiency, from automated alert enrichment and correlation to autonomous incident response, and the skills needed to manage these systems.
*   **Cloud-Native Security Operations:** Highlight the increasing importance of securing cloud environments (AWS, Azure, GCP) and the unique challenges and opportunities for SOC engineers in this domain.

**Main Section 2: Essential Skills for the High-Earning SOC Engineer of 2026**

*   **Deep Dive into Threat Hunting & Intelligence:** Emphasize the critical need for proactive threat hunting methodologies and the ability to effectively leverage and operationalize threat intelligence feeds.
*   **Mastery of Cloud Security Tools & Architectures:** Detail the necessity of understanding cloud security posture management (CSPM), cloud workload protection platforms (CWPP), and native cloud security services.
*   **Data Analytics & Scripting Prowess:** Explain the value of strong data analysis skills (SIEM, data lakes) and proficiency in scripting languages (Python, PowerShell) for automation and custom tool development.

**Main Section 3: Monetizing Your Expertise: Building a High-Income Stream**

*   **Specialization is Key:** Discuss focusing on in-demand niches like cloud security, incident response, threat intelligence analysis, or security automation engineering.
*   **Certifications that Pay Dividends:** Highlight industry-recognized certifications that demonstrate advanced skills and command higher salaries (e.g., CISSP, CCSP, GIAC certifications).
*   **Beyond the Full-Time Role:** Explore opportunities for high-income streams through consulting, freelance incident response, specialized training, or developing open-source security tools.

**Main Section 4: Cultivating the Future-Ready SOC Engineer Mindset**

*   **Continuous Learning is Non-Negotiable:** Stress the importance of staying ahead of emerging threats, technologies, and evolving best practices through ongoing education and research.
*   **Collaboration and Communication Excellence:** Explain how effective communication with development teams, management, and even legal/PR is crucial for successful incident management and influencing security strategy.
*   **Embrace the "Security as Code" Philosophy:** Advocate for treating security configurations and operations as code, enabling agility, repeatability, and scalability.

**Concluding Thought:** The SOC engineer of 2026 won't just be a guardian; they will be an architect, an analyst, and a strategic asset. By embracing the evolving SOC landscape, honing specialized skills, and actively pursuing income-generating opportunities, you can position yourself not just for job security, but for a truly high-income career in the vital field of cybersecurity. The future of defense is here – are you ready to lead it?
WriterAgent > ## Level Up Your SOC Skills: The High-Income Blueprint for Cybersecurity Engineers in 2026

The cybersecurity landscape is a battlefield, and the Security Operations Center (SOC) is on the front lines. But the tools and tactics of yesterday are no longer enough. As threats become more sophisticated, so too must the professionals defending our digital fortresses. If you're a SOC engineer looking to not just survive, but thrive and command a high income in 2026, you need to understand the seismic shifts happening in SOC evolution and strategically position yourself for success.

The SOC is rapidly evolving beyond simple alert triage. We're moving towards a proactive and predictive model, leveraging threat intelligence and advanced analytics to hunt for threats before they strike. The rise of Automation and AI/ML is transforming SOC efficiency, automating tasks from alert enrichment to incident response. Crucially, cloud-native security operations are no longer optional; mastering the nuances of securing AWS, Azure, and GCP environments is paramount.

To command a high income in 2026, you need a robust skill set. Deep dives into threat hunting methodologies and operationalizing threat intelligence are essential. Mastery of cloud security tools like CSPM and CWPP, alongside native cloud services, is non-negotiable. Furthermore, strong data analytics skills for SIEMs and data lakes, coupled with scripting prowess in Python or PowerShell, will set you apart for automation and custom tool development.

Monetizing your expertise requires strategic specialization. Focus on in-demand niches such as cloud security, incident response, threat intelligence analysis, or security automation engineering. Industry-recognized certifications like CISSP, CCSP, or advanced GIAC certifications will significantly boost your earning potential. Don't limit yourself to traditional roles; explore high-income streams through consulting, freelance incident response, specialized training, or even contributing to open-source security tools.

Cultivating a future-ready mindset is vital. Continuous learning is your greatest asset, keeping you ahead of emerging threats and technologies. Excellent collaboration and communication skills are crucial for effective incident management and influencing security strategy. Finally, embrace the "Security as Code" philosophy, treating security operations with the agility and scalability of software development.

The SOC engineer of 2026 will be an architect, an analyst, and a strategic asset. By embracing these shifts, honing specialized skills, and actively pursuing income-generating opportunities, you can secure not just job stability, but a truly high-income career in cybersecurity. The future of defense is here – are you ready to lead it?
EditorAgent > ## Level Up Your SOC Skills: The High-Income Blueprint for Cybersecurity Engineers in 2026

The cybersecurity landscape is a perpetual battlefield, with the Security Operations Center (SOC) standing firmly on the front lines. However, the tools and tactics that sufficed yesterday are rapidly becoming obsolete. As cyber threats grow increasingly sophisticated, so too must the professionals tasked with defending our digital fortresses. For SOC engineers aiming not just to survive but to thrive and command a high income by 2026, understanding the seismic shifts in SOC evolution and strategically positioning yourselves for success is paramount.

The SOC is undergoing a significant transformation, moving far beyond its traditional role of simple alert triage. We are witnessing a profound shift towards a proactive and predictive model. This evolution involves leveraging advanced threat intelligence and sophisticated analytics to actively hunt for threats *before* they can inflict damage. The pervasive rise of Automation and AI/ML is revolutionizing SOC efficiency, automating tasks ranging from alert enrichment and correlation to aspects of incident response. Crucially, cloud-native security operations are no longer a niche concern; mastering the intricacies of securing AWS, Azure, and GCP environments is now a non-negotiable requirement.

To command a high income in 2026, a robust and specialized skill set is essential. This includes deep dives into advanced threat hunting methodologies and the practical operationalization of threat intelligence feeds. Mastery of cloud security tools, such as Cloud Security Posture Management (CSPM) and Cloud Workload Protection Platforms (CWPP), alongside a solid understanding of native cloud security services, is critical. Furthermore, strong data analytics skills, particularly with SIEMs and data lakes, combined with scripting proficiency in languages like Python or PowerShell, will differentiate you significantly, enabling automation and custom tool development.

Monetizing your expertise effectively requires strategic specialization. Focus on high-demand niches such as cloud security architecture, advanced incident response, in-depth threat intelligence analysis, or security automation engineering. Earning industry-recognized certifications like the CISSP, CCSP, or advanced GIAC certifications will demonstrably boost your earning potential. Don't limit your income aspirations to traditional full-time roles; actively explore high-income streams through specialized consulting, freelance incident response engagements, developing and delivering specialized training programs, or even contributing meaningfully to open-source security tools.

Cultivating a future-ready mindset is equally vital. Continuous learning is your most potent asset, ensuring you remain ahead of emerging threats and evolving technologies. Exceptional collaboration and communication skills are indispensable for effective incident management and for influencing strategic security decisions across an organization. Finally, wholeheartedly embrace the "Security as Code" philosophy, treating security configurations and operational processes with the same agility, repeatability, and scalability expected in modern software development.

The SOC engineer of 2026 will transcend the role of a mere defender; they will be an architect of security, a keen analyst, and a strategic business asset. By embracing these critical shifts, honing specialized skills, and actively pursuing diverse income-generating opportunities, you can secure not only job stability but a truly high-income career within the indispensable field of cybersecurity. The future of digital defense is rapidly unfolding – are you ready to lead the charge?

👏 Great job! You've now created a reliable "assembly line" using a sequential agent, where each step runs in a predictable order.

**This is perfect for tasks that build on each other, but it's slow if the tasks are independent.** Next, we'll look at how to run multiple agents at the same time to speed up your workflow.

---
### Section 4
## 🛣️ Parallel Workflows - Independent Researchers

**The Problem: The Bottleneck**

The previous sequential agent is great, but it's an assembly line. Each step must wait for the previous one to finish. What if you have several tasks that are **not dependent** on each other? For example, researching three *different* topics. Running them in sequence would be slow and inefficient, creating a bottleneck where each task waits unnecessarily.

**The Solution: Concurrent Execution**

When you have independent tasks, you can run them all at the same time using a `ParallelAgent`. This agent executes all of its sub-agents concurrently, dramatically speeding up the workflow. Once all parallel tasks are complete, you can then pass their combined results to a final 'aggregator' step.

**Use Parallel when:** Tasks are independent, speed matters, and you can execute concurrently.

To learn more, check out the documentation related to [parallel agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/parallel-agents/).

**Architecture: Multi-Topic Research**

<!--
```mermaid
graph TD
    A["User Request: Research 3 topics"] -- > B["Parallel Execution"]
    B -- > C["Tech Researcher"]
    B -- > D["Health Researcher"]
    B -- > E["Finance Researcher"]

    C -- > F["Aggregator"]
    D -- > F
    E -- > F
    F -- > G["Combined Report"]

    style B fill:#ffffcc
    style F fill:#ffccff
```
-->

<img width="600" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/parallel-agent.png" alt="Parallel Agent" />

### 4.1 Example: Parallel Multi-Topic Research

Let's build a system with four agents:

1. **Tech Researcher** - Researches AI/ML news and trends
2. **Health Researcher** - Researches recent medical news and trends
3. **Finance Researcher** - Researches finance and fintech news and trends
4. **Aggregator Agent** - Combines all research findings into a single summary

In [12]:
# Tech Researcher: Focuses on AI in cybersecurity.
tech_researcher = Agent(
    name="TechResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research the latest AI/ML trends in cybersecurity. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research", # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [13]:
# Health Researcher: Focuses on medical breakthroughs.
health_researcher = Agent(
    name="HealthResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research", # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [14]:
# Finance Researcher: Focuses on fintech trends.
finance_researcher = Agent(
    name="FinanceResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research", # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


In [15]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model="gemini-2.5-flash-lite",
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary", # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


👉 **Then we bring the agents together under a parallel agent, which is itself nested inside of a sequential agent.**

This design ensures that the research agents run first in parallel, then once all of their research is complete, the aggregator agent brings together all of the research findings into a single report:

In [16]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


Let's run the agent and give it a prompt to research the given topics:

In [17]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Run the daily executive briefing on Tech, Health, and Finance")


 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
FinanceResearcher > **Fintech Trends Shaping the Financial Landscape**

The financial technology (fintech) sector is undergoing rapid transformation, driven by innovation and evolving consumer demands. Three key trends are currently defining its trajectory:

1.  **Embedded Finance and Open Banking:** This trend involves integrating financial services directly into non-financial platforms, such as e-commerce sites or apps. The market implications include a significant expansion of financial service accessibility and a blurring of lines between traditional finance and other industries. Projections indicate the embedded finance market could reach hundreds of billions of dollars by 2030.

2.  **Artificial Intelligence (AI) and Machine Learning (ML):** AI and ML are revolutionizing fintech by automating tasks, personalizing services, and enhancing fraud detection. The market imp

 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
FinanceResearcher > **Tech:**
Key trends include the surge in AI workloads demanding new infrastructure orchestration, the rise of "context engineering" for more reliable AI outputs, and the increasing adoption of AI agents for complex tasks. Market implications involve significant cloud infrastructure spending and evolving platform engineering roles. The future outlook points to AI becoming more integrated into workflows, demanding specialized hardware and sophisticated management.

**Health:**
Prominent trends are the expansion of personalized healthcare driven by AI and data, the growth of "Wearables 2.0" including BCIs and implants, and a strong focus on mental wellness technologies like VR/AR therapy and AI chatbots. Market implications see increased investment in health-tech and a greater demand for digital health solutions. The future anticipates more proactive, data-driven, and personalized health management.

**Finance:**
Major trends include the integration of AI and automation into financial systems for portfolio management and risk assessment, a wave of mega-mergers in banking (particularly in India), and increased focus on data-backed decision-making. Market implications involve a shift in demand for finance professionals with both financial modeling and digital transformation skills. The future outlook suggests continued AI-driven efficiency gains and structural changes in the global financial landscape.
TechResearcher > **AI in Cybersecurity: Key Trends and Impact**

The integration of Artificial Intelligence (AI) is rapidly transforming cybersecurity, offering enhanced defense mechanisms while simultaneously presenting new challenges. Key AI trends in cybersecurity include:

1.  **Advanced Threat Detection and Response:** AI algorithms can analyze vast datasets in real-time to identify anomalies, predict attacks, and automate incident responses. This leads to faster detection of both known and zero-day threats with fewer false positives. Companies like JPMorgan Chase utilize AI for fraud detection.
2.  **AI-Powered Cybercrime:** Malicious actors are leveraging AI to create more sophisticated and convincing attacks, such as hyper-realistic phishing emails and deepfakes. Generative AI, in particular, lowers the barrier to entry for cybercriminals, enabling them to develop advanced exploit code and conduct large-scale attacks more efficiently.
3.  **AI Agents and Multi-Agent Systems:** The rise of autonomous AI agents working collaboratively presents new attack vectors and vulnerabilities, requiring enhanced security measures for AI systems themselves. Protecting these systems from data poisoning and prompt injection is becoming a critical concern.

Major companies involved include Microsoft, Google, IBM, and Darktrace. The potential impact is significant, promising more efficient, effective, and secure defenses, but also demanding new strategies to counter AI-enhanced threats.
HealthResearcher > Here's a briefing on recent breakthroughs in Tech, Health, and Finance:

**Health:**
*   **mRNA Vaccine Technology:** Proven by COVID-19 vaccines, this technology allows rapid development of new vaccines for diseases like shingles and potentially cancer.
*   **AI in Healthcare:** Advanced AI is enhancing diagnostics, treatment planning, and even drug discovery.
*   **Lab-Grown Blood:** The successful transfusion of lab-grown red blood cells holds promise for treating blood disorders.

**Technology:**
*   **Agentic AI:** AI is evolving from assistive tools to autonomous agents capable of managing end-to-end workflows.
*   **Neural Interfaces (BCIs):** Brain-computer interfaces are moving from labs to practical applications, restoring mobility and enabling thought-controlled devices.
*   **5G Connectivity:** Continual expansion of 5G enables advancements in autonomous vehicles, smart cities, and remote healthcare.

**Finance:**
*   **Embedded Finance:** Financial services are increasingly integrated into non-financial platforms, projected to reach $138 billion by 2026.
*   **AI-Powered Personalization:** AI is driving personalized financial experiences, with the AI market in financial services already valued at $115.4 billion.
*   **Central Bank Digital Currencies (CBDCs):** CBDCs are progressing through pilots and launches, aiming to improve financial inclusion and transaction efficiency.

**Timelines:**
Most of these technologies are already in practical application or advanced development, with significant growth projected within the next 1-5 years.
AggregatorAgent > ## Executive Briefing: AI, Personalization, and Evolving Workflows Across Tech, Health, and Finance

Artificial Intelligence (AI) is the unifying force across technology, health, and finance, driving unprecedented advancements and new challenges. In **technology**, AI is powering agentic systems capable of complex, end-to-end workflows, necessitating robust infrastructure and specialized "context engineering" for reliable outputs. This evolution is mirrored in **health**, where AI enhances diagnostics, drug discovery, and treatment planning, leading to increasingly personalized and data-driven healthcare. Emerging technologies like mRNA vaccines and lab-grown blood further underscore the sector's rapid innovation.

Similarly, **finance** is leveraging AI for personalized experiences and sophisticated risk assessment, with embedded finance and CBDCs reshaping service delivery. A significant interconnected theme is the rise of autonomous AI agents, which, while promising efficiency, also introduce novel cybersecurity threats. Malicious actors are weaponizing AI for sophisticated attacks, while the security of AI systems themselves, particularly against prompt injection and data poisoning, is paramount. Across all sectors, expect continued integration of AI, a growing demand for specialized digital skills, and significant market growth within the next 1-5 years.

---
### Section 5
## ➰ Loop Workflows - The Refinement Cycle

**The Problem: One-Shot Quality**

All the workflows we've seen so far run from start to finish. The `SequentialAgent` and `ParallelAgent` produce their final output and then stop. This 'one-shot' approach isn't good for tasks that require refinement and quality control. What if the first draft of our story is bad? We have no way to review it and ask for a rewrite.

**The Solution: Iterative Refinement**

When a task needs to be improved through cycles of feedback and revision, you can use a `LoopAgent`. A `LoopAgent` runs a set of sub-agents repeatedly *until a specific condition is met or a maximum number of iterations is reached.* This creates a refinement cycle, allowing the agent system to improve its own work over and over.

**Use Loop when:** Iterative improvement is needed, quality refinement matters, or you need repeated cycles.

To learn more, check out the documentation related to [loop agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/loop-agents/).

**Architecture: Story Writing & Critique Loop**

<!--
```mermaid
graph TD
    A["Initial Prompt"] -- > B["Writer Agent"]
    B -- >|story| C["Critic Agent"]
    C -- >|critique| D{"Iteration < Max<br>AND<br>Not Approved?"}
    D -- >|Yes| B
    D -- >|No| E["Final Story"]

    style B fill:#ccffcc
    style C fill:#ffcccc
    style D fill:#ffffcc
```
-->

<img width="250" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/loop-agent.png" alt="Loop Agent" />

### 5.1 Example: Iterative Story Refinement

Let's build a system with two agents:

1. **Writer Agent** - Writes a draft of a short story
2. **Critic Agent** - Reviews and critiques the short story to suggest improvements

In [18]:
# This agent runs ONCE at the beginning to create the first draft.
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story", # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [19]:
# This agent's only job is to provide feedback or the approval signal. It has no tools.
critic_agent = Agent(
    name="CriticAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique", # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


Now, we need a way for the loop to actually stop based on the critic's feedback. The `LoopAgent` itself doesn't automatically know that "APPROVED" means "stop."

We need an agent to give it an explicit signal to terminate the loop.

We do this in two parts:

1. A simple Python function that the `LoopAgent` understands as an "exit" signal.
2. An agent that can call that function when the right condition is met.

First, you'll define the `exit_loop` function:

In [20]:
# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}

print("✅ exit_loop function created.")

✅ exit_loop function created.


To let an agent call this Python function, we wrap it in a `FunctionTool`. Then, we create a `RefinerAgent` that has this tool.

👉 **Notice its instructions:** this agent is the "brain" of the loop. It reads the `{critique}` from the `CriticAgent` and decides whether to (1) call the `exit_loop` tool or (2) rewrite the story.

In [21]:
# This agent refines the story based on critique OR calls the exit_loop function.
refiner_agent = Agent(
    name="RefinerAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    
    output_key="current_story", # It overwrites the story with the new, refined version.
    tools=[FunctionTool(exit_loop)], # The tool is now correctly initialized with the function reference.
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


Then we bring the agents together under a loop agent, which is itself nested inside of a sequential agent.

This design ensures that the system first produces an initial story draft, then the refinement loop runs up to the specified number of `max_iterations`:

In [22]:
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2, # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


Let's run the agent and give it a topic to write a short story about:

In [23]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a short story about an alert fatigue for SOC Analyst and steps to rectify using AI")


 ### Created new session: debug_session_id

User > Write a short story about an alert fatigue for SOC Analyst and steps to rectify using AI
InitialWriterAgent > The glowing monitors were a constant hum, a symphony of red and yellow alerts that Kai had learned to tune out. Another day, another deluge of false positives from the SIEM. His eyes burned, his mind felt like static. He’d clicked through hundreds of notifications before lunch, each one a tiny prick of anxiety that never bloomed into anything real. The truly critical alerts were probably lost in the noise, invisible until it was too late. He slumped back, the weight of it all crushing.

Later that week, a new AI-powered threat intelligence platform was integrated. It began by analyzing historical alerts, learning Kai’s established patterns and the typical false positive sources. Within days, the deluge slowed to a trickle of high-fidelity, actionable incidents. The AI flagged anomalies Kai would have missed, prioritizing genui

User > Write a short story about an alert fatigue for SOC Analyst and steps to rectify using AI
InitialWriterAgent > The blinking red lights on Elias’s dashboard were a familiar, nauseating hum. Another alert. And another. And another. He’d long stopped differentiating between a true threat and a network hiccup. His eyes, gritty from staring at the screen, scanned the torrent of notifications. Most were noise, a digital echo of the day’s routine. He’d clicked ‘false positive’ so many times the phrase had lost all meaning.

Then, a flicker of something different. Not the usual frantic pattern, but a subtle anomaly. If only he had the energy to investigate. He sighed, reaching for his coffee.

He remembered a new AI tool they'd piloted. It was supposed to learn his patterns, filter the noise. He’d been skeptical. He activated it. Within minutes, the cascade of alerts thinned. A single, high-priority notification remained, accompanied by a concise summary: "Suspicious outbound traffic, potential data exfiltration attempt. Originating from workstation X, user Y." Elias leaned forward, the fatigue momentarily forgotten. The AI had found the needle in the haystack.
CriticAgent > This is a solid start to a story that effectively captures the essence of alert fatigue and introduces a potential AI solution.

Here are a few suggestions for improvement:

1.  **Show, Don't Just Tell Elias's Fatigue:** While the story mentions his "nauseating hum" and "gritty eyes," it could be more impactful to *show* his fatigue through his actions and internal monologue. For instance, describe him rubbing his temples, the way his focus drifts, or a brief, cynical internal thought about the futility of his job before the AI intervenes. This would make his eventual re-engagement with the critical alert more dramatic.

2.  **Flesh Out the "Subtle Anomaly":** The transition from "noise" to a "subtle anomaly" is a crucial pivot. You could slightly expand on what made this anomaly different. Was it a unique combination of events? A timing issue? Even a brief descriptive phrase about the *nature* of the anomaly could heighten the suspense and make Elias's pause more significant.

3.  **Strengthen the AI's Impact:** The AI’s effectiveness is the story’s payoff. While the summary it provides is good, consider adding a brief sentence about *how* it arrived at that conclusion, or what Elias *would have had to do* manually to discover it. This would emphasize the AI's value beyond just filtering, showcasing its analytical capabilities and making Elias's relief more profound. For example, "The AI had not only filtered the noise but correlated seemingly unrelated events, a process that would have taken Elias hours of manual analysis."
RefinerAgent > The blinking red lights on Elias’s dashboard were a familiar, nauseating hum. Another alert. And another. And another. He rubbed his temples, the dull throb a constant companion to the screen’s glare. He’d long stopped differentiating between a true threat and a network hiccup. His eyes, gritty from staring at the endless cascade of notifications, scanned the torrent. Most were noise, a digital echo of the day’s mundane routine. He’d clicked ‘false positive’ so many times the phrase had lost all meaning, a Pavlovian response to the siren song of his job. *What’s the point?* he thought, his focus drifting to the cooling dregs of his coffee. *Just more data to sift through.*

Then, a flicker of something different. Not the usual frantic pattern, but a subtle anomaly. A brief, almost imperceptible dip in traffic, followed by an unusual spike in outbound requests from a non-standard port. It was the digital equivalent of a quiet cough in a silent room – easily missed, yet distinct. If only he had the energy to investigate. He sighed, reaching for his coffee, the lukewarm liquid doing little to cut through his weariness.

He remembered a new AI tool they'd piloted. It was supposed to learn his patterns, filter the noise, and flag genuine threats. He’d been skeptical, another piece of software promising miracles. But desperation gnawed at him. He activated it. Within minutes, the cascade of alerts thinned, the obnoxious red lights softening to a more manageable amber. A single, high-priority notification remained, accompanied by a concise, almost eerily calm summary: "Suspicious outbound traffic, potential data exfiltration attempt. Originating from workstation X, user Y. Correlation suggests a novel exploit targeting legacy systems." Elias leaned forward, the fatigue momentarily forgotten, replaced by a jolt of adrenaline. The AI had not only filtered the noise but correlated seemingly unrelated events, a process that would have taken him hours of painstaking manual analysis. It had found the needle in the haystack, and in doing so, had breathed life back into his flagging vigilance.
CriticAgent > APPROVED

You've now implemented a loop agent, creating a sophisticated system that can iteratively review and improve its own output. This is a key pattern for ensuring high-quality results.

You now have a complete toolkit of workflow patterns. Let's put it all together and review how to choose the right one for your use case.

--- 
### Section 6
## Summary - Choosing the Right Pattern

### Decision Tree: Which Workflow Pattern?

<!--
```mermaid
graph TD
    A{"What kind of workflow do you need?"} -- > B["Fixed Pipeline<br>(A → B → C)"];
    A -- > C["Concurrent Tasks<br>(Run A, B, C all at once)"];
    A -- > D["Iterative Refinement<br>(A ⇆ B)"];
    A -- > E["Dynamic Decisions<br>(Let the LLM decide what to do)"];

    B -- > B_S["Use <b>SequentialAgent</b>"];
    C -- > C_S["Use <b>ParallelAgent</b>"];
    D -- > D_S["Use <b>LoopAgent</b>"];
    E -- > E_S["Use <b>LLM Orchestrator</b><br>(Agent with other agents as tools)"];

    style B_S fill:#f9f,stroke:#333,stroke-width:2px
    style C_S fill:#ccf,stroke:#333,stroke-width:2px
    style D_S fill:#cff,stroke:#333,stroke-width:2px
    style E_S fill:#cfc,stroke:#333,stroke-width:2px
```
-->

<img width="1000" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/agent-decision-tree.png" alt="Agent Decision Tree" />

### Quick Reference Table

| Pattern | When to Use | Example | Key Feature |
|---------|-------------|---------|-------------|
| **LLM-based (sub_agents)** | Dynamic orchestration needed | Research + Summarize | LLM decides what to call |
| **Sequential** | Order matters, linear pipeline | Outline → Write → Edit | Deterministic order |
| **Parallel** | Independent tasks, speed matters | Multi-topic research | Concurrent execution |
| **Loop** | Iterative improvement needed | Writer + Critic refinement | Repeated cycles |

---

## ✅ Congratulations! You're Now an Agent Orchestrator

In this notebook, you made the leap from a single agent to a **multi-agent system**.

You saw **why** a team of specialists is easier to build and debug than one "do-it-all" agent. Most importantly, you learned how to be the **director** of that team.

You used `SequentialAgent`, `ParallelAgent`, and `LoopAgent` to create deterministic workflows, and you even used an LLM as a 'manager' to make dynamic decisions. You also mastered the "plumbing" by using `output_key` to pass state between agents and make them collaborative.

**ℹ️ Note: No submission required!**

This notebook is for your hands-on practice and learning only. You **do not** need to submit it anywhere to complete the course.

### 📚 Learn More

Refer to the following documentation to learn more:

- [Agents in ADK](https://google.github.io/adk-docs/agents/)
- [Sequential Agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/)
- [Parallel Agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/parallel-agents/)
- [Loop Agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/loop-agents/)
- [Custom Agents in ADK](https://google.github.io/adk-docs/agents/custom-agents/)

### 🎯 Next Steps

Ready for the next challenge? Stay tuned for Day 2 notebooks where we'll learn how to create **Custom Functions, use MCP Tools** and manage **Long-Running operations!**